In [1]:
import pandas as pd
import numpy as nm
import re
from tqdm.notebook import tqdm
import json
import time
secret_file = "data/secret_key.json"
with open(secret_file) as f:
    secret_key = json.loads(f.read())
    def get_secret_key(key, secret = secret_key):
        try:
            return secret[key]
        except KeyError:
            error_msg = f"Set the {key} environment variable"
            print(error_msg)
kakaoAK_REST_Key = get_secret_key("kakaoAK_REST_Key")
kakaoAK_REST_Key

'c227ddccaa0eb6add96fd0ebe203ae6d'

In [45]:
cctv_seoul = pd.read_csv("data/서울시 (안심이) CCTV 설치 현황_220615.csv")
column = ["구명", "주소", "CCTV용도", "lat", "lng", "CCTV수량", "수정일시"]
cctv_seoul.columns = column
cctv_seoul.head()

,구명,주소,CCTV용도,lat,lng,CCTV수량,수정일시
0,종로구,방범373 충신길 53 메인1,도심공원,37.575179,127.007360,1,2021-11-16
1,종로구,방범459 백석동길 187 검지2,도심공원,37.596453,126.967939,1,2021-11-16
2,종로구,방범128 돈화문로11다길 44 검지1,도심공원,37.573735,126.989771,1,2021-11-16
3,종로구,방범398 종로51가길 31 메인1,도심공원,37.573467,127.012433,1,2021-11-16
4,종로구,방범458 삼청로5길 30 검지3,도심공원,37.582970,126.980254,1,2021-11-16


In [8]:
import requests, json
from pprint import pprint
def get_address(lat, lng):
    url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x="+lng+"&y="+lat
    headers = {"Authorization": "KakaoAK " + kakaoAK_REST_Key}
    api_json = json.loads(str(requests.get(url, headers=headers).text))
    match_data = api_json["documents"][0]
    addr_dong = {"주소": match_data["address_name"], "법정동명": match_data["region_3depth_name"]}
    
    return addr_dong
cctv_addr = []
for n, m in tqdm(cctv_seoul_test[["lat", "lng"]].iterrows()):
    match_data = get_address(str(m.lat), str(m.lng))
    cctv_addr.append(match_data)


0it [00:00, ?it/s]

In [38]:
cctv_seoul_test = cctv_seoul
cctv_addr_df = pd.DataFrame(cctv_addr)
cctv_addr_nm = cctv_addr_df["주소"].str.split(" ", expand=True)
cctv_addr_nm.rename(columns={1: "구명"}, inplace=True)
cctv_addr_gudong = pd.concat([cctv_addr_nm["구명"], cctv_addr_df], axis = 1)
cctv_addr_gudong.drop("주소", axis = 1, inplace=True)
cctv_addr_gudong

,구명,법정동명
0,종로구,충신동
1,종로구,부암동
2,종로구,익선동
3,종로구,창신동
4,종로구,팔판동
...,...,...
59419,강남구,역삼동
59420,강남구,논현동
59421,강남구,논현동
59422,강남구,역삼동


In [43]:
cctv_seoul_test = cctv_seoul_test[["CCTV수량", "lat", "lng"]]
cctv_seoul_latlng = pd.concat([cctv_addr_gudong, cctv_seoul_test], axis = 1)
cctv_seoul_latlng.head()
cctv_seoul_latlng["CCTV수량"].value_counts()

1     54544
3      1631
2      1385
4      1094
5       601
6       155
7         4
8         3
9         2
19        1
28        1
14        1
10        1
11        1
Name: CCTV수량, dtype: int64

In [42]:
cctv_seoul_latlng.to_csv("data/서울시CCTV좌표.csv")

In [ ]:
police_substation = pd.read_csv("data/경찰청_전국 경찰관서 현황_20211231.csv", encoding="cp949")
police_substation_seoul = police_substation.loc[police_substation["전체기관명"].str.contains("서울")]
name_list = police_substation_seoul["전체기관명"].str.split(" ", expand=True)
name_list.drop(0, inplace=True, axis =1)
name_list.columns = ["경찰청명", "지역구경찰서명", "관서명"]
police_substation_seoul = pd.concat([police_substation_seoul, name_list], axis = 1)
police_substation_seoul.reset_index(drop=True, inplace=True)
police_substation_seoul

In [ ]:
tmp = police_substation_seoul["도로명주소"]
police_substation_seoul_df = pd.DataFrame(tmp)
police_substation_seoul_df.reset_index(drop=True, inplace = True)
police_address_list = police_substation_seoul_df["도로명주소"].str.split(" ", expand=True)
police_address_list

In [ ]:
import requests, json
import time
police_address_latlng = []
def get_location(location):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query=' + location
    headers = {"Authorization": "KakaoAK c227ddccaa0eb6add96fd0ebe203ae6d"}
    api_json = json.loads(str(requests.get(url,headers=headers).text))
    address = api_json['documents']
    for addr_nm in address:
        if "서울" in addr_nm["address_name"]:
            crd = {"lat": str(addr_nm['y']), "lng": str(addr_nm['x']), "주소": addr_nm["address_name"]}
            return crd  

for i, location in police_substation_seoul.iterrows():
    if location.관서명:
        crd = get_location(location.관서명)
        police_address_latlng.append(crd)
    elif location.관서명 == None:
        if location.지역구경찰서명:
            crd = get_location(location.지역구경찰서명)
            police_address_latlng.append(crd)
        else:
            crd = get_location(location.경찰청명)
            police_address_latlng.append(crd)

pprint(police_address_latlng)

In [ ]:
police_address_latlng_df = pd.DataFrame(police_address_latlng)
police_addr_split = police_address_latlng_df["주소"].str.split(" ", expand=True)
police_latlng = police_addr_split.loc[:, 1:2]
police_latlng.columns = ["구명", "법정동명"]
police_latlng_df = pd.concat([police_latlng, police_address_latlng_df], axis = 1)
police_latlng_df

In [ ]:
police_latlng_df.to_csv("data/서울파출소_지구대_좌표.csv")

In [ ]:
pd.reset_option("display.max.row")